In [1]:
print_imports()

Loaded modules:
numpy                np              1.14.0
pandas               pd              0.22.0
sklearn              sk              0.19.1
keras                ks              2.1.4

matplotlib           mpl             2.1.2
matplotlib.pyplot    plt             N/A
matplotlib.image     mpimg           N/A
seaborn              sns             0.8.1
PIL                  PIL             5.0.0

ExergyUtilities      exergy          2.0.

pyspark              pyspark         2.2.1


In [2]:
import os, json, logging
from pprint import pprint

In [3]:
root_path = r"/home/batman/git/ref_DataScienceRetreat/DSR Lecture notebooks/Module X - Practical Aspects - Patrick/data/fraud-data"

In [4]:
files = []
dfs=list()
for fname in os.listdir(root_path):

    data = []
    with open(os.path.join(root_path,fname),'r') as f:
        for line in f:
            data.append(json.loads(line))
        
    files.append(data)
    dfs.append(pd.DataFrame(data))


In [5]:
df = pd.concat(dfs)
df.head()

,basket,customerID,fraudLabel,totalAmount,zipCode
0,"[1, 3, 0, 3, 3, 3]",2945531578,0,186,12975
1,[1],7923206372,0,57,82971
2,"[0, 3]",7164247248,0,128,56553
3,[4],6040018816,0,27,80182
4,"[3, 1, 0]",3879244510,0,123,76967


In [6]:
df.describe()

,customerID,fraudLabel,totalAmount,zipCode
count,3.100000e+04,31000.000000,31000.000000,31000.000000
mean,5.534906e+09,0.029839,237.825935,54727.113871
std,2.602565e+09,0.170145,204.574924,25909.857668
min,1.000031e+09,0.000000,10.000000,10004.000000
25%,3.298389e+09,0.000000,77.000000,32143.000000
50%,5.545815e+09,0.000000,183.000000,54695.500000
75%,7.791624e+09,0.000000,340.000000,77205.500000
max,9.999907e+09,1.000000,2064.000000,100000.000000


Duplicates?

In [7]:
sum(df['customerID'].duplicated())
#df[customerID.isin(customerID[customerID.duplicated()])].sort("customerID")

0

FEATURE - Length of basket

In [8]:
df['num_items'] = df['basket'].apply(len)
df.head(2)

,basket,customerID,fraudLabel,totalAmount,zipCode,num_items
0,"[1, 3, 0, 3, 3, 3]",2945531578,0,186,12975,6
1,[1],7923206372,0,57,82971,1


FEATURE - One hot basket

In [9]:
%%script false 
#Alternative!!!
df['c_0'] = df.basket.map(lambda x: x.count(0))

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
#mlb = MultiLabelBinarizer()
dfd = df.join(pd.DataFrame(mlb.fit_transform(df.pop('basket')),
                          columns=mlb.classes_,
                          index=df.index))
#df2.head()

In [11]:
df.head()

,customerID,fraudLabel,totalAmount,zipCode,num_items
0,2945531578,0,186,12975,6
1,7923206372,0,57,82971,1
2,7164247248,0,128,56553,2
3,6040018816,0,27,80182,1
4,3879244510,0,123,76967,3


In [12]:
#s = df['basket'].apply(lambda x: pd.Series(x)).unstack()
# df2 = df.join(pd.DataFrame(s.reset_index(level=0, drop=True))).drop('basket',1).rename(columns={0:'basket'})
# df2.merge(pd.get_dummies(df2['basket']), left_index=True, right_index=True).drop('basket',1)
# df2.head()

FEATURE - One hot zip code - Very large and sparse

In [13]:
df = pd.get_dummies(df,columns=['zipCode'])
df.head(2)

,customerID,fraudLabel,totalAmount,num_items,zipCode_10004,zipCode_10006,zipCode_10011,zipCode_10016,zipCode_10019,zipCode_10021,...,zipCode_99975,zipCode_99978,zipCode_99979,zipCode_99980,zipCode_99985,zipCode_99988,zipCode_99993,zipCode_99994,zipCode_99999,zipCode_100000
0,2945531578,0,186,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7923206372,0,57,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#df_zipcode = pd.get_dummies(df,columns=['zipCode'])
#df_zipcode.head(2)

In [15]:
len(df.columns)

25960

X and y data

In [16]:
y = df['fraudLabel']
#cols = ['num_items','totalAmount']
#cols = ['totalAmount']
X=df#[cols]

X.head(2)

,customerID,fraudLabel,totalAmount,num_items,zipCode_10004,zipCode_10006,zipCode_10011,zipCode_10016,zipCode_10019,zipCode_10021,...,zipCode_99975,zipCode_99978,zipCode_99979,zipCode_99980,zipCode_99985,zipCode_99988,zipCode_99993,zipCode_99994,zipCode_99999,zipCode_100000
0,2945531578,0,186,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7923206372,0,57,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#print(files[0])
#pprint(files[0])
#print(len())

In [18]:
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split

/home/batman/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Fit a simple model

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf_0 = LogisticRegression().fit(X_train, y_train)

pred_y_0 = clf_0.predict(X_test)

In [20]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf_0.score(X_test, pred_y_0)))

Accuracy of logistic regression classifier on test set: 1.00


In [29]:
print( np.unique( pred_y_0 ) )

[0]


Gradient Boosting Classifier

In [22]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

#X, y = make_hastie_10_2(random_state=0)
#X_train, X_test = X[:2000], X[2000:]
#y_train, y_test = y[:2000], y[2000:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

clf_1 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

In [30]:
pred_y_1 = clf_1.predict(X_test)

In [31]:
print(np.unique(pred_y_1))

[0 1]


In [24]:
clf_1.score(X_test, y_test)             

1.0

In [25]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf_1.score(X_test, y_test)))
#clf_1.score(X_test, pred_y_1))

Accuracy of logistic regression classifier on test set: 1.00


In [ ]:
%%script false
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators': 500, 'max_depth': 6,
        'learning_rate': 0.1, 'loss': 'huber','alpha':0.95}
clf_1 = GradientBoostingRegressor(**params).fit(X, y)
pred_y_1 = clf_1.predict(X)

New model fit

In [ ]:
# Predict class probabilities
prob_y_2 = clf_2.predict_proba(X)
 
# Keep only the positive class
prob_y_2 = [p[1] for p in prob_y_2]
 

In [ ]:
# Predict class probabilities
prob_y_2 = logreg.predict_proba(X)
 
# Keep only the positive class
prob_y_2 = [p[1] for p in prob_y_2]
 

In [ ]:
prob_y_2[:5]

In [ ]:
print( roc_auc_score(y, prob_y_2) )

In [ ]:
df['fraudLabel'].value_counts()